In [1]:
%config InlineBackend.figure_format='retina'

# Add the parent directory to the path so that we can import the modules.
from functools import partial
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from market_conditions import *
from dynamic_contagion import *
from reinsurance import *

import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Other parameters
maturity = 3
markup = 0.4
R = int(1e5)
seed = 123

# Asset parameters
V_0 = 130
phi_V = -3  # * (1.3) # = V0 / L0
sigma_V = 0.05

# Liability parameters
L_0 = 100
phi_L = -3
sigma_L = 0.02

# Interest rate parameters
r_0 = 0.02
k = 0.2
m = 0.05  # 0.5
upsilon = 0.1
eta_r = -0.01

# Catastrophe loss size distribution parameters
mu_C = 2
sigma_C = 0.5

In [3]:
# Poisson process
def simulate_poisson(seed):
    lambda_ = 0.5
    rg = rnd.default_rng(seed)
    return rg.poisson(lambda_ * maturity)


# Cox proces
def simulate_cox(seed):
    lambda0 = 0.49
    a = 0.4
    rho = 0.4
    delta = 1

    return simulate_num_dynamic_contagion_uniform_jumps(
        seed, maturity, lambda0, a, rho, delta, 0.0, 0.0, 0.0, 0.5
    )


# Hawkes process
def simulate_hawkes(seed):
    lambda0 = 0.47
    a = 0.26
    rho = 0.4
    delta = 1

    return simulate_num_dynamic_contagion_uniform_jumps(
        seed, maturity, lambda0, a, rho, delta, 0.0, 1.0, 0.0, 0.0
    )


# Dynamic contagion process
def simulate_dcp(seed):
    lambda0 = 0.29
    a = 0.26
    rho = 0.4
    delta = 1

    return simulate_num_dynamic_contagion_uniform_jumps(
        seed, maturity, lambda0, a, rho, delta, 0.0, 1.0, 0.0, 0.5
    )

## Tables 1-4

In [4]:
%%time
prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    V_0,
    L_0,
    r_0,
    (simulate_poisson, simulate_cox, simulate_hawkes, simulate_dcp),
    mu_C,
    sigma_C,
    markup,
    As=(10.0, 15.0, 20.0, 25.0, 30.0),
    Ms=(60.0, 65.0, 70.0, 75.0, 80.0, 85.0, 90.0),
)

prices.shape

Loading 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=100000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=130,L_0=100,r_0=0.02.npy'


100%|██████████| 4/4 [00:16<00:00,  4.04s/it]

CPU times: total: 8.52 s
Wall time: 16.8 s


(4, 5, 7)

In [5]:
# Tables 1 to 4
cat_models = ("Poisson", "Cox", "Hawkes", "DCP")
for c in range(4):
    print(cat_models[c])
    display(pd.DataFrame(prices[c]).round(4))

Poisson


,0,1,2,3,4,5,6
0,7.2478,7.2561,7.2605,7.2625,7.2635,7.2640,7.2643
1,4.5758,4.5848,4.5895,4.5916,4.5927,4.5932,4.5935
2,2.7822,2.7921,2.7972,2.7995,2.8006,2.8012,2.8015
3,1.6393,1.6500,1.6557,1.6581,1.6594,1.6600,1.6603
4,0.9403,0.9519,0.9580,0.9607,0.9620,0.9627,0.9630


Cox


,0,1,2,3,4,5,6
0,7.2451,7.2538,7.2583,7.2607,7.2620,7.2626,7.2629
1,4.6032,4.6126,4.6175,4.6201,4.6215,4.6221,4.6224
2,2.8205,2.8309,2.8362,2.8390,2.8405,2.8412,2.8415
3,1.6718,1.6831,1.6890,1.6920,1.6936,1.6944,1.6947
4,0.9628,0.9750,0.9813,0.9846,0.9863,0.9872,0.9876


Hawkes


,0,1,2,3,4,5,6
0,8.1183,8.1950,8.2488,8.2876,8.3151,8.3339,8.3476
1,5.9402,6.0235,6.0817,6.1233,6.1525,6.1726,6.1871
2,4.3240,4.4150,4.4781,4.5231,4.5545,4.5758,4.5912
3,3.1259,3.2249,3.2938,3.3428,3.3766,3.3995,3.4158
4,2.2317,2.3390,2.4140,2.4674,2.5043,2.5289,2.5464


DCP


,0,1,2,3,4,5,6
0,8.2255,8.3003,8.3523,8.3891,8.4147,8.4329,8.4457
1,6.0212,6.1026,6.1588,6.1981,6.2253,6.2447,6.2582
2,4.3848,4.4738,4.5349,4.5773,4.6064,4.6270,4.6414
3,3.1672,3.2644,3.3310,3.3771,3.4085,3.4305,3.4458
4,2.2562,2.3615,2.4342,2.4843,2.5184,2.5421,2.5585


## Table 5: Default risk premium

In [6]:
%%time
risky_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    V_0,
    L_0,
    r_0,
    (simulate_poisson, simulate_cox, simulate_hawkes, simulate_dcp),
    mu_C,
    sigma_C,
    markup,
)[:, np.newaxis]

Loading 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=100000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=130,L_0=100,r_0=0.02.npy'


100%|██████████| 4/4 [00:13<00:00,  3.26s/it]


CPU times: total: 8.09 s
Wall time: 14.7 s


In [7]:
risky_prices.round(4)

array([[2.8015],
       [2.8415],
       [4.5912],
       [4.6414]])

In [8]:
%%time
safe_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    V_0,
    L_0,
    r_0,
    (simulate_poisson, simulate_cox, simulate_hawkes, simulate_dcp),
    mu_C,
    sigma_C,
    markup,
    defaultable=False,
)[:, np.newaxis]

Loading 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=100000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=130,L_0=100,r_0=0.02.npy'


100%|██████████| 4/4 [00:04<00:00,  1.09s/it]

CPU times: total: 1.91 s
Wall time: 5.73 s


In [9]:
safe_prices.round(4)

array([[2.847 ],
       [2.8844],
       [4.8709],
       [4.9161]])

In [10]:
risk_premium = safe_prices - risky_prices
risk_premium.round(4)

array([[0.0455],
       [0.0429],
       [0.2797],
       [0.2747]])

In [11]:
np.hstack((risky_prices, safe_prices, risk_premium)).round(4)

array([[2.8015, 2.847 , 0.0455],
       [2.8415, 2.8844, 0.0429],
       [4.5912, 4.8709, 0.2797],
       [4.6414, 4.9161, 0.2747]])

## Table 6: Impacts of externally-excited jump frequency rate

In [12]:
def simulate_dcp_variations(seed, rho):
    lambda0 = 0.29
    a = 0.26
    delta = 1

    return simulate_num_dynamic_contagion_uniform_jumps(
        seed, maturity, lambda0, a, rho, delta, 0.0, 1.0, 0.0, 0.5
    )


simulators = [partial(simulate_dcp_variations, rho=rho) for rho in (0.4, 3, 10, 20)]

In [13]:
%%time
risky_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    tuple(int(scale * L_0) for scale in (1.1, 1.3, 1.5)),
    L_0,
    r_0,
    simulators,
    mu_C,
    sigma_C,
    markup,
).T

Loading 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=100000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=110,L_0=100,r_0=0.02.npy'


100%|██████████| 4/4 [00:04<00:00,  1.08s/it]


Loading 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=100000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=130,L_0=100,r_0=0.02.npy'


100%|██████████| 4/4 [00:04<00:00,  1.05s/it]


Loading 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=100000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=150,L_0=100,r_0=0.02.npy'


100%|██████████| 4/4 [00:04<00:00,  1.05s/it]


CPU times: total: 3.47 s
Wall time: 14.8 s


In [14]:
risky_prices.round(4)

array([[ 4.1942,  4.6414,  4.8367],
       [13.593 , 15.3399, 16.2959],
       [39.9647, 46.4996, 51.4606],
       [56.7379, 66.8978, 76.0682]])

In [15]:
%%time
safe_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    tuple(int(scale * L_0) for scale in (1.1, 1.3, 1.5)),
    L_0,
    r_0,
    simulators,
    mu_C,
    sigma_C,
    markup,
    defaultable=False,
).T

Loading 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=100000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=110,L_0=100,r_0=0.02.npy'


100%|██████████| 4/4 [00:04<00:00,  1.02s/it]

CPU times: total: 1.22 s
Wall time: 4.69 s


In [16]:
safe_prices.round(4)

array([[ 4.9161,  4.9161,  4.9161],
       [16.7538, 16.7538, 16.7538],
       [54.7041, 54.7041, 54.7041],
       [83.286 , 83.286 , 83.286 ]])

In [17]:
risk_premium = safe_prices - risky_prices

In [18]:
np.hstack([risky_prices, risk_premium]).round(4)

array([[ 4.1942,  4.6414,  4.8367,  0.7219,  0.2747,  0.0794],
       [13.593 , 15.3399, 16.2959,  3.1608,  1.414 ,  0.4579],
       [39.9647, 46.4996, 51.4606, 14.7394,  8.2045,  3.2435],
       [56.7379, 66.8978, 76.0682, 26.5481, 16.3882,  7.2178]])

## Table 7: Impacts of externally-excited jump magnitude

In [19]:
def simulate_dcp_variations(seed, mu_F=0.25, mu_G=0.5):
    lambda0 = 0.29
    a = 0.26
    delta = 1
    rho = 0.4

    return simulate_num_dynamic_contagion_uniform_jumps(
        seed, maturity, lambda0, a, rho, delta, 0.0, 2 * mu_G, 0.0, 2 * mu_F
    )


simulators = [
    partial(simulate_dcp_variations, mu_F=mu_F) for mu_F in (0.25, 1.0, 4.0, 8.0)
]

In [20]:
%%time
risky_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    tuple(int(scale * L_0) for scale in (1.1, 1.3, 1.5)),
    L_0,
    r_0,
    simulators,
    mu_C,
    sigma_C,
    markup,
).T

safe_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    tuple(int(scale * L_0) for scale in (1.1, 1.3, 1.5)),
    L_0,
    r_0,
    simulators,
    mu_C,
    sigma_C,
    markup,
    defaultable=False,
).T

risk_premium = safe_prices - risky_prices

Loading 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=100000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=110,L_0=100,r_0=0.02.npy'


100%|██████████| 4/4 [00:03<00:00,  1.26it/s]


Loading 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=100000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=130,L_0=100,r_0=0.02.npy'


100%|██████████| 4/4 [00:03<00:00,  1.23it/s]


Loading 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=100000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=150,L_0=100,r_0=0.02.npy'


100%|██████████| 4/4 [00:03<00:00,  1.22it/s]


Loading 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=100000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=110,L_0=100,r_0=0.02.npy'


100%|██████████| 4/4 [00:03<00:00,  1.33it/s]

CPU times: total: 3.92 s
Wall time: 15.2 s


In [21]:
np.hstack([risky_prices, risk_premium]).round(4)

array([[ 4.1942,  4.6414,  4.8367,  0.7219,  0.2747,  0.0794],
       [ 8.599 ,  9.6724, 10.253 ,  1.9358,  0.8625,  0.2818],
       [23.4755, 27.2457, 30.1638,  8.7104,  4.9402,  2.0221],
       [31.6212, 37.0257, 41.6676, 13.6175,  8.213 ,  3.5712]])

## Table 8: Impacts of self-excited jump magnitude

In [22]:
%%time

simulators = [
    partial(simulate_dcp_variations, mu_G=mu_G) for mu_G in (0.5, 1.0, 2.0, 3.0)
]

risky_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    tuple(int(scale * L_0) for scale in (1.1, 1.3, 1.5)),
    L_0,
    r_0,
    simulators,
    mu_C,
    sigma_C,
    markup,
).T

safe_prices = reinsurance_prices(
    R,
    seed,
    maturity,
    k,
    eta_r,
    m,
    phi_V,
    sigma_V,
    phi_L,
    sigma_L,
    upsilon,
    tuple(int(scale * L_0) for scale in (1.1, 1.3, 1.5)),
    L_0,
    r_0,
    simulators,
    mu_C,
    sigma_C,
    markup,
    defaultable=False,
).T

risk_premium = safe_prices - risky_prices

Loading 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=100000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=110,L_0=100,r_0=0.02.npy'


100%|██████████| 4/4 [00:03<00:00,  1.06it/s]


Loading 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=100000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=130,L_0=100,r_0=0.02.npy'


100%|██████████| 4/4 [00:03<00:00,  1.05it/s]


Loading 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=100000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=150,L_0=100,r_0=0.02.npy'


100%|██████████| 4/4 [00:03<00:00,  1.14it/s]


Loading 'c:\Users\Patrick\Dropbox\Jiwook Project Local\catastrophe-reinsurance-pricing-compound-point-process\data\mc-R=100000,seed=123,maturity=3,k=0.2,lambda_r=-0.01,m=0.05,phi_V=-3,sigma_V=0.05,phi_L=-3,sigma_L=0.02,upsilon=0.1,V_0=110,L_0=100,r_0=0.02.npy'


100%|██████████| 4/4 [00:03<00:00,  1.15it/s]

CPU times: total: 4.62 s
Wall time: 17 s


In [23]:
np.hstack([risky_prices, risk_premium]).round(4)

array([[ 4.1942,  4.6414,  4.8367,  0.7219,  0.2747,  0.0794],
       [ 9.1782, 10.4914, 11.379 ,  2.7508,  1.4376,  0.5499],
       [19.0579, 22.2719, 25.0156,  8.0597,  4.8457,  2.102 ],
       [24.9325, 29.305 , 33.245 , 11.4795,  7.1069,  3.1669]])